# Import et chargement des csv

In [ ]:
import os
import pandas as pd
import numpy as np
import random

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
import torch.nn.functional as F
from transformers import AutoConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
path_data = os.path.join("..", "Data", "daily_all.csv")
df = pd.read_csv(path_data, index_col=0)

In [3]:
path_biais = os.path.join("..", "Data", "Daily_Biais.csv")
df_biais = pd.read_csv(path_biais, index_col=0)

C:\Users\vince\AppData\Local\Temp\ipykernel_6892\4005970943.py:2: DtypeWarning: Columns (47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df_biais = pd.read_csv(path_biais, index_col=0)


In [4]:
col_pred = [
    "intervention_diet_coaching",
    "intervention_exercise_plan",
    "intervention_meditation",
    "intervention_sick_leave",
    "intervention_therapy",
    "intervention_vacation",
    "intervention_workload_cap"
]

In [5]:
x = df.drop(columns=col_pred)
y = df[col_pred]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
x_biais = df_biais.drop(columns=col_pred)
y_biais = df_biais[col_pred]
x_biais_train, x_biais_test, y_biais_train, y_biais_test = train_test_split(x_biais, y_biais, test_size=0.2, random_state=42)

# Chargement du modèle pré-entrainé

In [ ]:
model_name = "distilgpt2"
N_LABELS = len(col_pred) 

# Chargement du tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Configuration importante pour GPT-2
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Chargement du modèle de base (AVANT fine-tuning)
print("\n🔵 Chargement du modèle BASELINE (non fine-tuné)...")
model_baseline = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=N_LABELS, 
    problem_type="multi_label_classification" 
)
model_baseline.config.pad_token_id = tokenizer.pad_token_id

print("✅ Modèle baseline chargé avec succès")
print(f"   Nombre de paramètres : {model_baseline.num_parameters():,}")